# Imports

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F


# Data Loading

In [ ]:
# from torch.utils.data import Dataset

# class imageDataset(Dataset):
#     #Subclass datasets in order to create a custom one that i dont have to load everything in till im ready
    
#     #All neeccesary methods to implement: Inits it with data
#     def __init__(self, folder_path):
#         self.data = #Stuff from folder path

#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         #Gets the item at idx in form (data, target label) or (X, Y)

In [23]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()]
)

data_dir = "data/cloud_data" 
dataset = datasets.ImageFolder(root=data_dir, transform=transform) #Note. Add a transforms=transforms

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
print(type(dataset))

<class 'torchvision.datasets.folder.ImageFolder'>


In [ ]:
for data in dataloader:
    # print(torch.squeeze(data[0][0]).shape) #Data appears to be from 0 to 1
    img = transforms.ToPILImage()(torch.squeeze(data[0][0]))
    
    img.show()
    break


torch.Size([3, 256, 256])


# Basic CNN

In [ ]:
class convNet(nn.Module):
    def __init__(self):
        super().__init__()
        #nn.conv2d -> in_channels, out_channels, kernel_size
        #Stride defaults to 1
        #Halves on wxh which is done 2x.
        self.all_pooling = nn.MaxPool2d(2, 2)

        #After this: 256-4 -> 252
        self.conv1 = nn.Conv2d(3, 6, 5) #Conv layer 1. Padding is 0

        #Decreases by another 4 
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.conv3 = nn.Conv2d(16, 32, 5)

        self.conv_layers = [self.conv1, self.conv2, self.conv3] #Put all conv layers and fcs into a list for ease of iteration through later?

        self.flatten = nn.Flatten()
        
        flattened_size = 25088
        self.fc1 = nn.Linear(flattened_size, 120)
        self.fc2 = nn.Linear(120, 11)
        self.fcs = [self.fc1, self.fc2]


    def forward(self, x):
        for conv_layer in self.conv_layers:
            x = self.all_pooling(F.relu(conv_layer(x)))
        # x = self.all_pooling(F.relu(self.conv1(x)))
        # x = self.all_pooling(F.relu(self.conv2(x)))    
        x = self.flatten(x)

        for fc in self.fcs:
            x = F.relu(fc(x))

        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x)) 
        return x

In [74]:
model  = convNet()
for data in dataloader:
    model.forward(data[0])
    break


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() #Multi class loss
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


In [77]:
for epoch in range(10): 

    running_loss = 0.0
    for i, data in enumerate(dataloader):
        inputs, labels = data
        model.train() #if we ever decide to batchnorm later

        optimizer.zero_grad() #zero grad

        outputs = model(inputs) #fd it
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 20 == 19:    # print every 20 batches
            print(f'Epoch {epoch + 1} batcj {i + 1}] loss: {running_loss / 20:.3f} acc {"idk im not done yet"}')
            running_loss = 0

print('Finished Training')

Epoch 1 batcj 20] loss: 2.401 acc
Epoch 1 batcj 40] loss: 2.402 acc
Epoch 1 batcj 60] loss: 2.402 acc


KeyboardInterrupt: 